<a href="https://colab.research.google.com/github/bryantjr98/Portfolio/blob/main/SalesForcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%pip install -Uq upgini catboost

In [18]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=12_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,2,3,19
1,2013-01-01,7,5,5
2,2013-01-01,10,21,33
3,2013-01-01,3,27,11
4,2013-01-01,4,9,19


In [19]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [20]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [21]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
              eval_set=[(test_features, test_target)])

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample




Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=65079b30-64d1-4d71-b512-ac818983945c
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


39 relevant feature(s) found with the search keys: ['date']


f_autofe_div_0a5adf97,1.1183,100.0000,"0.1392, 0.0645, -0.3926",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_536f8702,1.1164,100.0000,"-0.9231, 0.8369, -0.7584",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_efd66bbe,1.0686,100.0000,"0.9351, 0.4315, -0.5716",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_1edd5009,1.0472,100.0000,"-0.0091, 0.0563, 0.0231",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_34d11bc4,0.9956,100.0000,"-16.6059, 5.3999, -2.9504",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_cbf285e0,0.7026,100.0000,"-0.6576, 1.1045, 0.2626",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_25296268,0.6455,100.0000,"-1.1257, 0.6987, -0.8516",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_04d9bc60,0.6416,100.0000,"-0.6906, -0.9368, -0.9375",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_b49258bb,0.5871,100.0000,"120.9581, 347.4935, -256.3704",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_b20799dd,0.5855,100.0000,"1.0857, 0.9871, 1.0014",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_5388a2c0,0.4933,100.0000,"0.6963, 1.6174, 0.8481",Upgini,AutoFE: features from Markets data,Daily


Upgini,"AutoFE: features from Calendar data,Markets data",11.3018,23
Upgini,AutoFE: features from Calendar data,1.4208,8
Upgini,AutoFE: features from Markets data,1.3385,3
Upgini,Calendar data,0.0092,4
Upgini,World economic indicators,0.0019,1


"Calendar data,Markets data",f_autofe_mul_efd66bbe,f_events_date_year_cos1_9014a856,f_financial_date_vix_7d_to_1y_634c77eb,*
"Calendar data,Markets data",f_autofe_div_1edd5009,f_events_date_year_cos1_9014a856,f_financial_date_vix_6c8e6194,/
"Calendar data,Markets data",f_autofe_div_0a5adf97,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_92dac942,/
"Calendar data,Markets data",f_autofe_div_cbf285e0,f_events_date_year_cos1_9014a856,f_financial_date_gold_7d_to_7d_1y_shift_a8127a43,/
"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_div_536f8702,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_1y_7a6f300d,/
"Calendar data,Markets data",f_autofe_div_0c0eef24,f_events_date_week_cos3_7525fe31,f_financial_date_gold_7d_to_7d_1y_shift_a8127a43,/
"Calendar data,Markets data",f_autofe_div_cf5dd7be,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,/
"Calendar data,Markets data",f_autofe_mul_89f64ca9,f_events_date_week_sin1_847b5db1,f_financial_date_natural_gas_7d_to_1y_7a6f300d,*
Markets data,f_autofe_div_0dcebab1,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,f_financial_date_snp500_7d_to_1y_378b9f99,/
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/


We detected 64 outliers in your sample.
Examples of outliers with maximum value of target:
45    205
25    196
60    180
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,9578,50.2328,306.9850,233.1561,73.8289
Eval 1,2422,59.6908,501.9799,423.4478,78.5321


In [22]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
      train_features, train_target,
      eval_set=[(test_features, test_target)],
      estimator=model,
      scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


f_autofe_div_1edd5009,1.3113,100.0000,"-0.0091, 0.0563, 0.0231",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_34d11bc4,1.1739,100.0000,"-16.6059, 5.3999, -2.9504",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_536f8702,1.0919,100.0000,"-0.9231, 0.8369, -0.7584",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_efd66bbe,0.9742,100.0000,"0.9351, 0.4315, -0.5716",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_3109d259,0.8943,100.0000,"-0.899, -0.7324, -0.7406",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_0a5adf97,0.8775,100.0000,"0.1392, 0.0645, -0.3926",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_4a9fa180,0.7437,100.0000,"-0.0688, -0.3013, 0.0607",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_mul_b49258bb,0.7126,100.0000,"120.9581, 347.4935, -256.3704",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_5388a2c0,0.7117,100.0000,"0.6963, 1.6174, 0.8481",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_cbf285e0,0.6968,100.0000,"-0.6576, 1.1045, 0.2626",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_0dcebab1,0.5766,100.0000,"1.0451, 1.0308, 1.0405",Upgini,AutoFE: features from Markets data,Daily


Upgini,"AutoFE: features from Calendar data,Markets data",11.8373,23
Upgini,AutoFE: features from Calendar data,2.3033,8
Upgini,AutoFE: features from Markets data,1.8285,3
Upgini,Calendar data,0.0092,4
Upgini,World economic indicators,0.0019,1


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,9578,50.2328,0.254443,0.174464,0.079979
1,Eval 1,2422,59.6908,0.266192,0.213787,0.052405


In [23]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.
WARNING #3: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history




Column name,Status,Errors
date,All valid,-



Running search request, search_id=2975dee5-e033-4260-9c67-50a8d349d996
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.
WARNING #4: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history




Column name,Status,Errors
date,All valid,-



Running search request, search_id=ddbe2034-2703-41d4-89b9-d3d2539a237f
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_autofe_div_1edd5009,f_autofe_mul_34d11bc4,f_autofe_div_536f8702,f_autofe_mul_efd66bbe,f_events_date_year_cos1_9014a856,f_autofe_div_3109d259,f_autofe_div_0a5adf97,...,f_autofe_mul_89f64ca9,f_autofe_div_c855cf40,f_autofe_mul_cd42199d,f_autofe_mul_af6d166b,f_autofe_div_22b77c5b,f_autofe_div_ba625fe9,f_autofe_mul_6a97c336,f_autofe_mul_041b52f4,f_autofe_mul_0afc7563,f_autofe_mul_7fc70d87
0,2013-01-01,2,3,0.054674,29.727047,0.817451,1.132905,0.98522,0.733748,0.294008,...,0.94229,0.007816,-3.019147,-1.036025,0.847837,0.79356,0.716951,0.133922,0.770276,0.614275
1,2013-01-01,7,5,0.054674,29.727047,0.817451,1.132905,0.98522,0.733748,0.294008,...,0.94229,0.007816,-3.019147,-1.036025,0.847837,0.79356,0.716951,0.133922,0.770276,0.614275
2,2013-01-01,10,21,0.054674,29.727047,0.817451,1.132905,0.98522,0.733748,0.294008,...,0.94229,0.007816,-3.019147,-1.036025,0.847837,0.79356,0.716951,0.133922,0.770276,0.614275
3,2013-01-01,3,27,0.054674,29.727047,0.817451,1.132905,0.98522,0.733748,0.294008,...,0.94229,0.007816,-3.019147,-1.036025,0.847837,0.79356,0.716951,0.133922,0.770276,0.614275
4,2013-01-01,4,9,0.054674,29.727047,0.817451,1.132905,0.98522,0.733748,0.294008,...,0.94229,0.007816,-3.019147,-1.036025,0.847837,0.79356,0.716951,0.133922,0.770276,0.614275


In [24]:
model.fit(train_features,train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[40.984044429997745]

In [25]:
model.fit(enriched_train_features,train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")

[14.888256669748532]